In [1]:
import requests
from bs4 import BeautifulSoup

In [194]:
def extract_text_from_cf_problem_page(problem_url):
    problem_html = requests.get(problem_url)
    problem_parse = BeautifulSoup(problem_html.content,"html.parser")
    problem_div = problem_parse.find_all("div")
    #問題文の抽出,コドフォのページのhtmlをよく見て該当部分の属性で抽出
    res={"problem_text":[],"input_text":[],"output_text":[],"tags":[]}
    for elem in problem_div:
        try:
            elem_cls = elem.get("class")[0]
            if(elem_cls == "input-specification"):
                res["input_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])
            if(elem_cls == "output-specification"):
                res["output_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])
                break
        except:
            if(not res["problem_text"]):
                res["problem_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])

    res["problem_text"]="".join(res["problem_text"])
    res["input_text"]="".join(res["input_text"])
    res["output_text"]="".join(res["output_text"])
    #タグの抽出,コドフォのhtmlをよく見て該当部分の属性で抽出
    problem_span = problem_parse.find_all("span")
    for elem in problem_span:
        try:
            elem_cls = elem.get("class")[0]
            if(elem_cls == "tag-box"):
                res["tags"].append(elem.string)
        except:
            pass
    return res

In [195]:
extract_text_from_cf_problem_page("https://codeforces.com/problemset/problem/1492/B")

<span class="tag-box" style="font-size:1.2rem;" title="Heaps, binary search trees, segment trees, hash tables, etc">
    data structures
</span>
<span class="tag-box" style="font-size:1.2rem;" title="Greedy algorithms">
    greedy
</span>
<span class="tag-box" style="font-size:1.2rem;" title="Mathematics including integration, differential equations, etc">
    math
</span>
<span class="tag-box" style="font-size:1.2rem;" title="Difficulty">
    *1100
</span>


{'problem_text': "You have a deck of $$$n$$$ cards, and you'd like to reorder it to a new one.Each card has a value between $$$1$$$ and $$$n$$$ equal to $$$p_i$$$. All $$$p_i$$$ are pairwise distinct. Cards in a deck are numbered from bottom to top, i.\xa0e. $$$p_1$$$ stands for the bottom card, $$$p_n$$$ is the top card. In each step you pick some integer $$$k > 0$$$, take the top $$$k$$$ cards from the original deck and place them, in the order they are now, on top of the new deck. You perform this operation until the original deck is empty. (Refer to the notes section for the better understanding.)Given the original deck, output the deck with maximum possible order you can make using the operation above.The first line contains a single integer $$$t$$$ ($$$1 \\le t \\le 1000$$$)\xa0— the number of test cases.The first line of each test case contains the single integer $$$n$$$ ($$$1 \\le n \\le 10^5$$$)\xa0— the size of deck you have.The second line contains $$$n$$$ integers $$$p_1, p